In [2]:
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

In [3]:
seed = 7
numpy.random.seed(seed)

In [4]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 975s 6us/step


In [7]:
X_train.shape

(50000, 3, 32, 32)

In [8]:
# normalize inputs from 0-255 to 0.0-1.0
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [10]:
# one hot encode
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [11]:
y_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [18]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(3, 32, 32), padding='valid', activation='relu'))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='valid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='valid'))
model.add(Dropout(0.1))
model.add(Conv2D(32, (3, 3), activation='relu', padding='valid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
# Compile model
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 32, 30, 30)        896       
_________________________________________________________________
dropout_11 (Dropout)         (None, 32, 30, 30)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 32, 28, 28)        9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 32, 14, 14)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 32, 12, 12)        9248      
_________________________________________________________________
dropout_12 (Dropout)         (None, 32, 12, 12)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 32, 10, 10)        9248      
__________

In [19]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=100)

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 46s 921us/step - loss: 2.0137 - acc: 0.2585 - val_loss: 1.7435 - val_acc: 0.3793
Epoch 2/25
50000/50000 [==============================] - 46s 925us/step - loss: 1.6717 - acc: 0.3905 - val_loss: 1.5393 - val_acc: 0.4624
Epoch 3/25
50000/50000 [==============================] - 45s 903us/step - loss: 1.4802 - acc: 0.4636 - val_loss: 1.3821 - val_acc: 0.5066
Epoch 4/25
50000/50000 [==============================] - 46s 914us/step - loss: 1.3769 - acc: 0.5051 - val_loss: 1.2720 - val_acc: 0.5408
Epoch 5/25
50000/50000 [==============================] - 46s 911us/step - loss: 1.3035 - acc: 0.5328 - val_loss: 1.2170 - val_acc: 0.5653
Epoch 6/25
50000/50000 [==============================] - 44s 883us/step - loss: 1.2438 - acc: 0.5586 - val_loss: 1.1574 - val_acc: 0.5924
Epoch 7/25
50000/50000 [==============================] - 45s 894us/step - loss: 1.1867 - acc: 0.5792 - val_loss: 1.

In [20]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 70.69%
